In [ ]:
#### INSTALLATION OF LIBRARIES FOR SPARK
!pip install findspark
!sudo apt update
!sudo apt install openjdk-17-jdk -y
!curl -JLO 'https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz'
!tar xvf spark-3.3.1-bin-hadoop3.tgz
!mv spark-3.3.1-bin-hadoop3 /opt/spark
!pip install pyspark
!pip install pytorch-ignite transformers
print('Setup Complete');

### Load Dataset

In [ ]:
# !pip install gdown
# !mkdir data
# !cd data
# !gdown 1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
# !mv 'Dataset Yelp.zip' data.zip
# !unzip data.zip
# !mv  -v /notebooks/data/Dataset\ Yelp/* /notebooks/data/
# !rm -rf /notebooks/data/Dataset\ Yelp

In [55]:
#### SPARK
import findspark
findspark.init()
from pyspark.sql import SparkSession
#import pyspark.pandas as ps

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.3")\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF");

In [3]:
AllData = spark.read \
      .option("header", True) \
      .json("./")

AllData = AllData.select("text")
AllData.show(truncate=15)

+---------------+
|           text|
+---------------+
|If you decid...|
|I've taken a...|
|Family diner...|
|Wow!  Yummy,...|
|Cute interio...|
|I am a long ...|
|Loved this t...|
|Amazingly am...|
|This easter ...|
|Had a party ...|
|My experienc...|
|Locals recom...|
|Love going h...|
|Good food--l...|
|The bun make...|
|Great place ...|
|Tremendous s...|
|The hubby an...|
|I go to blow...|
|My absolute ...|
+---------------+
only showing top 20 rows



In [62]:
from transformers import pipeline
# if you have GPU access, you can set the device parameter to 0 to use the GPU, which will speed up model performance.
classifier = pipeline("sentiment-analysis", model="VictorSanh/roberta-base-finetuned-yelp-polarity", device=0)

Some weights of the model checkpoint at VictorSanh/roberta-base-finetuned-yelp-polarity were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import pyspark.pandas as ps
reviews = ps.read_json("./data/review.json", lines=True)

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [79]:
reviews_tiny = ps.DataFrame(reviews.loc[0:7000, "text"])

In [80]:
reviews_tiny['length'] = reviews_tiny['text'].apply(lambda x: len(str(x).split(' ')))

In [81]:
reviews_tiny['length'].describe()

count    7001.000000
mean      101.653764
std        92.241852
min         3.000000
25%        41.000000
50%        74.000000
75%       129.000000
max      1006.000000
Name: length, dtype: float64

In [56]:
reviews_tiny.head()

,text
0,"If you decide to eat here, just be aware it is..."
1,I've taken a lot of spin classes over the year...
2,Family diner. Had the buffet. Eclectic assortm...
3,"Wow! Yummy, different, delicious. Our favo..."
4,Cute interior and owner (?) gave us tour of up...


In [72]:
reviews_tiny.shape

(7001, 1)

In [75]:
def classify(x):
    x = x.values.tolist()
    return classifier(x).pop(0).get('label')#, classifier(x).pop(0).get('score')

In [76]:
%%time
#reviews_tiny['label'], reviews_tiny['score'] = reviews_tiny.apply(classify)
reviews_tiny['label'] = reviews_tiny.apply(classify)

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


RuntimeError: The expanded size of the tensor (788) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 788].  Tensor sizes: [1, 514]